In [1]:
import os
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
from shark_turbine.aot import *
from iree.compiler.ir import Context
from datasets import load_dataset
import PIL

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Load feature extractor and pretrained model from huggingface
extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

In [ ]:
###
#standard inference example (no SHARK-Turbine):
###

#load an example
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

#image.save("cats-image.jpg") #if you want to see the cute picture :)

#image2 = PIL.JpegImagePlugin.JpegImageFile("yourexamplepicture.jpg")

In [ ]:
#extract features from image(s) to feed to model
inputs = extractor(image, return_tensors="pt")
#inputs2 = extractor(image2, return_tensors="pt")

In [ ]:
#standard torch application of model to extracted feature input.
with torch.no_grad():
    logits = model(**inputs).logits
#    logits2 = model(**inputs2).logits

predicted_label = logits.argmax(-1).item()
#predicted_label2 = logits2.argmax(-1).item()

print(model.config.id2label[predicted_label])
#print(model.config.id2label[predicted_label2])

In [ ]:
###
#Using SHARK-Turbine to write a torch-mlir file for performing inference with the model.
###

#Questions: what is the minimum necessary functionality to include in a CompiledModule subclass? 

class wtfamidoing(CompiledModule):
    params = export_parameters(model, initialize =False) #                is this necessary? 
    
    @jittable
    def forward(image0):
        input0 = extractor(image0, return_tensors='pt')
        with torch.no_grad():
            logits0 = model(**input0).logits #                            Do I need to do anything else with input0?
        predicted_label0 = logits.argmax(-1).item()
        str_output = model.config.id2label[predicted_label0]
        return str_output

In [ ]:
#build an instance of our CompiledModule and get the corresponding mlir code
inst = wtfamidoing(context=Context(), import_to="IMPORT")
module_str = str(CompiledModule.get_mlir_module(inst))

In [ ]:
#Output a torch-ir mlir file
with open("resnet-18.mlir", "w+") as f:
    f.write(module_str)